### To run this code, initially files need to be linked to the location of notebook.

In [ ]:
with open('dummy_order.html', 'r') as f:
    html_content = f.read()
    #print(html_content)

In [ ]:
with open('dummy_shipping.html', 'r') as f:
    shipping = f.read()
    #print(shipping)

### Clean the data from html files.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

def clean_email_text(email_text):
    # Remove email headers, footers, and signatures using regular expressions
    pattern = r'From:.*|To:.*|Subject:.*|\n.*\nOn.*\n.*\n.*\nSent.*\n|Dear.*|Sincerely.*'
    email_text = re.sub(pattern, '', email_text, flags=re.IGNORECASE)
    email_text = re.sub('<[^<]+?>', '', email_text)
    email_text = email_text.replace('nbsp', ' ')

    # Convert the text to lowercase
    email_text = email_text.lower()

    # Remove punctuation marks
    email_text = re.sub(r'[^\w\s]', '', email_text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    
    words = nltk.word_tokenize(email_text)
    words = [word for word in words if word not in stop_words]
    email_text = ' '.join(words)

    return email_text

order = clean_email_text(html_content)
shipping = clean_email_text(shipping)
order


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'barnes amp noble order confirmation 241300967 find store view browser save 10 every day become member save 10 every daybecome member books teens kids ebooks toys games order summary order 241300967 ship 500 elkview drive miami fl 33169 us bill james smith shipping method standard item qty price est delivery fine print lauren asher paperback 1 1499 10212022 twisted love special edition twisted series 1 ana huang paperback 1 1399 10212022 ice planet barbarians ruby dixon paperback 1 1440 10222022 sub total 4338 shipping amp processing 000 tax 345 order total 4683 visa ending xxxx 4683 order contains preorder items available publication dates digital preorders appear nook nook reading app publication dates nook magazine newspaper subscriptions qualify 14day trial wont billed trial ends order delivery expectations best process ship orders quickly possible estimated delivery date cost shipping order depends selected delivery speed information please visit delivery expectations items order 

### Order data

In [ ]:
order

'barnes amp noble order confirmation 241300967 find store view browser save 10 every day become member save 10 every daybecome member books teens kids ebooks toys games order summary order 241300967 ship 500 elkview drive miami fl 33169 us bill james smith shipping method standard item qty price est delivery fine print lauren asher paperback 1 1499 10212022 twisted love special edition twisted series 1 ana huang paperback 1 1399 10212022 ice planet barbarians ruby dixon paperback 1 1440 10222022 sub total 4338 shipping amp processing 000 tax 345 order total 4683 visa ending xxxx 4683 order contains preorder items available publication dates digital preorders appear nook nook reading app publication dates nook magazine newspaper subscriptions qualify 14day trial wont billed trial ends order delivery expectations best process ship orders quickly possible estimated delivery date cost shipping order depends selected delivery speed information please visit delivery expectations items order 

### Shipping data

In [ ]:
shipping

'billing summary recent barnes amp noble order 262434598 view browser books ebooks audiobooks teens amp ya kids toys hi james summary listed items shippedbilled qty amount fine print 1 1499 order subtotal 1499 tax 100 order total 1599 prices shown reflect discounts applied applicable please note items order shipped yet bill items ship receive another email detailing charges payment cards chargedrefunded credit card ending xxxx 1599 order included items check status please note transactions bncom appear credit card statement barnesampnoble papersource thank shopping us look forward next visit barnes amp noble check order status learn easy returns questions visit bampn help center 2022 barnes amp noble rights reserved privacy policy bncom 33 east 17th street new york ny 10003'

### Tokenization

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer


# Tokenize the email text into words
#words = nltk.word_tokenize(order)
words = nltk.word_tokenize(shipping)

# Define a TF-IDF vectorizer and fit it to the words
tfidf = TfidfVectorizer()
tfidf.fit_transform(words)

# Extract the top 5 most important words and phrases
top_n = 5
top_words = sorted(list(zip(tfidf.get_feature_names_out(), tfidf.idf_)), key=lambda x: x[1])[:top_n]
top_phrases = []
for i in range(len(words)-1):
    phrase = f"{words[i]} {words[i+1]}"
    if phrase in tfidf.get_feature_names_out():
        top_phrases.append((phrase, tfidf.idf_[tfidf.get_feature_names_out().index(phrase)]))
top_phrases = sorted(top_phrases, key=lambda x: x[1])[:top_n]

# Print the top words and phrases
print("Top words:")
for word in top_words:
    print(word[0])
print("Top phrases:")
for phrase in top_phrases:
    print(phrase[0])


Top words:
order
amp
items
barnes
noble
Top phrases:


In [ ]:
order

'barnes amp noble order confirmation 241300967 find store view browser save 10 every day become member save 10 every daybecome member books teens kids ebooks toys games order summary order 241300967 ship 500 elkview drive miami fl 33169 us bill james smith shipping method standard item qty price est delivery fine print lauren asher paperback 1 1499 10212022 twisted love special edition twisted series 1 ana huang paperback 1 1399 10212022 ice planet barbarians ruby dixon paperback 1 1440 10222022 sub total 4338 shipping amp processing 000 tax 345 order total 4683 visa ending xxxx 4683 order contains preorder items available publication dates digital preorders appear nook nook reading app publication dates nook magazine newspaper subscriptions qualify 14day trial wont billed trial ends order delivery expectations best process ship orders quickly possible estimated delivery date cost shipping order depends selected delivery speed information please visit delivery expectations items order 

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Analyze the sentiment of the email text
sentiment = analyzer.polarity_scores(order)

# Print the sentiment scores
print("Sentiment scores:")
for key, value in sentiment.items():
    print(f"{key}: {value}")


Sentiment scores:
neg: 0.018
neu: 0.812
pos: 0.17
compound: 0.9935


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
sentiment = analyzer.polarity_scores(shipping)

# Print the sentiment scores
print("Sentiment scores:")
for key, value in sentiment.items():
    print(f"{key}: {value}")

Sentiment scores:
neg: 0.015
neu: 0.775
pos: 0.21
compound: 0.9738


# if the positivity score is more then it is likely to be shipping email as the email arrives like good news your items are shipped, whereas order receipts will be like confirmation of order and says thank you for placing the order



In [ ]:
order

'barnes amp noble order confirmation 241300967 find store view browser save 10 every day become member save 10 every daybecome member books teens kids ebooks toys games order summary order 241300967 ship 500 elkview drive miami fl 33169 us bill james smith shipping method standard item qty price est delivery fine print lauren asher paperback 1 1499 10212022 twisted love special edition twisted series 1 ana huang paperback 1 1399 10212022 ice planet barbarians ruby dixon paperback 1 1440 10222022 sub total 4338 shipping amp processing 000 tax 345 order total 4683 visa ending xxxx 4683 order contains preorder items available publication dates digital preorders appear nook nook reading app publication dates nook magazine newspaper subscriptions qualify 14day trial wont billed trial ends order delivery expectations best process ship orders quickly possible estimated delivery date cost shipping order depends selected delivery speed information please visit delivery expectations items order 

In [ ]:
shipping

'billing summary recent barnes amp noble order 262434598 view browser books ebooks audiobooks teens amp ya kids toys hi james summary listed items shippedbilled qty amount fine print 1 1499 order subtotal 1499 tax 100 order total 1599 prices shown reflect discounts applied applicable please note items order shipped yet bill items ship receive another email detailing charges payment cards chargedrefunded credit card ending xxxx 1599 order included items check status please note transactions bncom appear credit card statement barnesampnoble papersource thank shopping us look forward next visit barnes amp noble check order status learn easy returns questions visit bampn help center 2022 barnes amp noble rights reserved privacy policy bncom 33 east 17th street new york ny 10003'

In [ ]:
if 'confirmation' in order:
  print('this is order email')
else:
  print('this is shipping email')

this is order email


In [ ]:
if 'confirmation' in shipping:
  print('this is order email')
else:
  print('this is shipping email')

this is shipping email
